In [195]:
from typing import List, Text, Tuple, Union
from pprint import pprint

from pyvi import ViTokenizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sentence_transformers import SentenceTransformer

from va.src.utils.data import load_va_data

## Load Data

In [218]:
PATH = "../data/trungquan/trainset.xlsx"
X, y = load_va_data(PATH)

## Preprocess Text

In [219]:
def preprocess(texts):
    """ Word Segmentation """
    tokenize = lambda x: ViTokenizer.tokenize(x)

    return [tokenize(text) for text in texts]

In [220]:
X = preprocess(X)

## Encode Label

In [221]:
lb = LabelBinarizer()
y = lb.fit_transform(y)
pprint(lb.classes_)
pprint(y)

array(['C2A_TUVAN', 'C2B_GLS', 'C3_KQT'], dtype='<U9')
array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])


## Text Embedding

### TF-IDF

In [222]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))

In [223]:
tfidf_X = tfidf.fit_transform(X)

In [224]:
len(tfidf.vocabulary_)

1281

### Vietnamese SBERT

In [229]:
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')

In [230]:
sbert_X = sbert.encode(X)

In [231]:
sbert_X.shape

(396, 768)

## Classifier

### Logistic Regression

In [235]:
clf = {}
clf["tfidf"] = OneVsRestClassifier(LogisticRegression()).fit(tfidf_X, y)
clf["sbert"] = OneVsRestClassifier(LogisticRegression()).fit(sbert_X, y)

In [239]:
""" Inference """
def infer(text: Text,
          embedding_type: Text = "tfidf",
          with_prob: bool = False
    ) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
    pred = []

    if not isinstance(text, List):
        text = [text]

    if embedding_type == "tfidf":
        vectorizer = tfidf
        _clf = clf["tfidf"]
    elif embedding_type == "sbert":
        vectorizer = sbert
        _clf = clf["sbert"]
    else:
        raise ValueError

    # preprocess
    text = preprocess(text)
    # vectorize
    if embedding_type == "tfidf":
        features = vectorizer.transform(text)
    elif embedding_type == "sbert":
        features = vectorizer.encode(text)
    else:
        raise ValueError
    # classify
    pred = _clf.predict(features)

    if with_prob:
        prob = _clf.predict_proba(features)
        prob = np.max(prob, axis=1)
        return pred, prob

    return pred

#### TF-IDF Features

In [242]:
def test1(text):
    pred, prob = infer(text, embedding_type="tfidf", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [243]:
text = "em có thể diễn giải chi tiết hơn được không"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.34704396])


In [246]:
text = "hay là chút nữa liên lạc lại em nhé"
test1(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.51182048])


In [248]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test1(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.62048898])


In [251]:
text = "anh nghĩ là em nên dừng lại"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46300921])


In [253]:
text = "dạ không ạ"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46125805])


In [257]:
text = "không đâu chị ơi"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.36902174])


In [265]:
text = "ừ không, mình đang làm rồi bạn nha"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.38223752])


In [268]:
text = "dạ"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.48442125])


#### SBERT Features

In [244]:
def test2(text):
    pred, prob = infer(text, embedding_type="sbert", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [245]:
text = "em có thể diễn giải chi tiết hơn được không"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.69315568])


In [247]:
text = "hay là chút nữa liên lạc lại em nhé"
test2(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.95827348])


In [249]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.77097445])


In [252]:
text = "anh nghĩ là em nên dừng lại"
test2(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.21407058])


In [256]:
text = "dạ không ạ"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.89481114])


In [258]:
text = "không đâu chị ơi"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.66558446])


In [267]:
text = "ừ không, mình đang làm rồi bạn nha"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.80773914])
